<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/Most_Popular_Item_Recommender/Book_Crossing_MPIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>Non-personalized Recommendation Systems</h1></center>

---

<center>
  <img src="https://storage.googleapis.com/kaggle-datasets-images/1661575/2726067/684ac0c4c14cb46d1047ccb620b45cac/dataset-cover.jpg?t=2021-10-21-03-18-09" width="800" height="300">
</center>


## Objetivo de este Notebook

En este notebook, construiremos un sistema de recomendación no personalizado basado en los ítems más populares (Most-Popular-Item Recommender, MPIR). Además, evaluaremos la efectividad del sistema utilizando métricas como el Hit Rate, la Precisión y la Diversidad.

En la segunda parte, introduciremos un sistema semi-personalizado utilizando información demográfica.


## Tabla de Contenidos

1. <a>Introducción a los Sistemas de Recomendación</a>
2. <a>Contexto del Dataset</a>
3. <a>Descargar y preparar el Dataset</a>
4. <a>Pre-selección y tratamiento de variables</a>
5. <a>Construcción del modelo MPIR</a>
6. <a>Evaluación del modelo (Hit Rate, Precisión y Diversidad)</a>
7. <a>Semi-Personalización del Recomendador</a>
8. <a>Desafíos Adicionales</a>
9. <a>Conclusiones</a>

---


## 1. Introducción a los Sistemas de Recomendación

Los sistemas de recomendación se han convertido en una herramienta esencial en la era digital, ayudando a los usuarios a navegar por vastas cantidades de información al sugerir productos, servicios o contenidos que son relevantes para ellos. Los sistemas de recomendación se pueden clasificar en tres categorías principales:
<br><br>

<b>1.1. Recomendación no personalizada:</b><br>
- Se basa en características generales de los ítems, sin tener en cuenta las preferencias individuales de los usuarios. Ejemplo: recomendaciones basadas en los ítems más populares.

<b>1.2. Recomendación basada en contenido:</b><br>
- Se enfoca en recomendar ítems similares a aquellos que el usuario ha interactuado anteriormente. Ejemplo: recomendaciones de libros basadas en el género o autor preferido por el usuario.

<b>1.3. Recomendación colaborativa:</b><br>
- Se basa en las preferencias de otros usuarios similares. Si usuario similares han disfrutado de un libro, es probable que también te guste.

<br>En este laboratorio, implementaremos un sistema de recomendación no personalizada utilizando el enfoque de ítems más populares (MPIR).

---

## 2. Contexto del Dataset

El conjunto de datos "Book-Crossing" es una colección de información relacionada con libros, usuarios y calificaciones. Este dataset es útil para construir sistemas de recomendación, analizar patrones de lectura y preferencias de los usuarios. Incluye tres componentes principales:
<br><br>

<b>2.1. Libros: </b><br>
- Información sobre los libros, incluyendo su título, autor y año de publicación.

<b>2.2. Usuarios: </b><br>
- Perfiles de los usuarios que interactúan con los libros, incluyendo su ID, ubicación y edad.

<b>2.3. Calificaciones: </b><br>
- Calificaciones numéricas que los usuarios asignan a los libros que han leído.

<br>Nuestro objetivo es utilizar este dataset para construir un sistema de recomendación no personalizado que sugiere libros basados en su popularidad.

---

## 3. Descargar y preparar el Dataset


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Crear la carpeta llamada 'dataset'
!mkdir -p dataset

# Descargar los archivos CSV y guardarlos en la carpeta 'dataset'
!wget -O dataset/BX-Book-Ratings.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Book-Ratings.csv
!wget -O dataset/BX-Books.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Books.csv
!wget -O dataset/BX-Users.csv https://raw.githubusercontent.com/bigsnarfdude/guide-to-data-mining/master/BX-Dump/BX-Users.csv


In [4]:
# Cargar el dataset
ratings = pd.read_csv("dataset/BX-Book-Ratings.csv", sep=";", encoding="ISO-8859-1", header=None, na_values='\\N')
books = pd.read_csv("dataset/BX-Books.csv", sep=";", encoding="ISO-8859-1", header=None, na_values='\\N', on_bad_lines='skip')
users = pd.read_csv("dataset/BX-Users.csv", sep=";", encoding="ISO-8859-1", header=None, na_values='\\N')

# Asignar manualmente los nombres de las columnas
ratings.columns = ['User-ID', 'ISBN', 'Book-Rating']
books.columns = ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
users.columns = ['User-ID', 'Location', 'Age']


In [ ]:
# Previsualizar los datos
print("  Usuarios: {} \n  Libros: {}\n  Calificaciones: {}".format(len(users), len(books), len(ratings)))

In [ ]:
users.head()

In [ ]:
books.head()

In [ ]:
ratings.head()

---

## 4. Pre-selección y tratamiento de variables


En esta sección, prepararemos los datos antes de construir el modelo. Esto incluye la limpieza de los datos, la remoción de outliers, y la transformación de variables para asegurar que los datos estén en un formato adecuado.

In [9]:
# Limpieza de nombres de columnas
users.columns = users.columns.str.lower().str.replace('-', '_')
books.columns = books.columns.str.lower().str.replace('-', '_')
ratings.columns = ratings.columns.str.lower().str.replace('-', '_')

### 4.1. Tratamiento de datos de usuarios


Vamos a eliminar outliers en la variable de edad para evitar que valores extremos afecten el análisis. Esto es importante porque los outliers pueden distorsionar las recomendaciones al representar comportamientos que no son típicos.

In [ ]:
# Crear un boxplot
fig, ax1 = plt.subplots(figsize=(6, 2))
sns.boxplot(data=users[['age']], orient="h", ax=ax1)
ax1.set_title('Variable Original')

In [15]:
# Remoción de outliers en la variable de edad
# Cálculo del IQR
IQR = np.nanpercentile(users['age'], 75) - np.nanpercentile(users['age'], 25)

# Definir los umbrales inferior y superior para capear los outliers
lower_threshold = np.nanpercentile(users['age'], 25) - 1.5 * IQR
upper_threshold = np.nanpercentile(users['age'], 75) + 1.5 * IQR

# Capear los valores fuera de los umbrales
users['age'] = np.clip(users['age'], lower_threshold, upper_threshold)

In [ ]:
# Crear un boxplot
fig, ax1 = plt.subplots(figsize=(6, 2))
sns.boxplot(data=users[['age']], orient="h", ax=ax1)
ax1.set_title('Variable Tratada')

In [ ]:
# Visualización de la distribución de edades
plt.figure(figsize=(8, 4))
ax = sns.countplot(data=users, x='age', color='lightblue')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=8)
plt.xlabel('Edad', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.title('Distribución de Edad de usuarios', fontsize=14)
plt.tight_layout()
plt.show()


### 4.2. Tratamiento de datos de libros


En este paso, limpiaremos los datos de los libros, eliminando columnas irrelevantes y manejando valores incorrectos en el año de publicación. La eliminación de columnas innecesarias y la corrección de valores atípicos asegura que las recomendaciones se basen en datos fiables y relevantes.

In [18]:
# Eliminación de columnas irrelevantes (URLs de imágenes)
books.drop(columns=['image_url_s', 'image_url_m', 'image_url_l'], inplace=True)


In [ ]:
books.head()

In [20]:
# Convertir año de publicación a numérico y manejo de valores incorrectos
books['year_of_publication'] = pd.to_numeric(books['year_of_publication'], errors='coerce')
books['year_of_publication'].replace(0, np.nan, inplace=True)

In [ ]:
books.head()

In [ ]:
# Visualización de la distribución de edades
plt.figure(figsize=(12, 4))
ax = sns.countplot(data=books, x='year_of_publication', color='lightblue')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=8)
plt.xlabel('year_of_publication', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.title('Distribución de year_of_publication', fontsize=14)
plt.tight_layout()
plt.show()


In [27]:
# Remoción de outliers en año de publicación
books = books[(books['year_of_publication'] >= 1964) & (books['year_of_publication'] <= 2004)]
books['year_of_publication'] = books['year_of_publication'].astype(int)

In [ ]:
# Visualización de la distribución de años de publicación
plt.figure(figsize=(10, 4))
ax = sns.countplot(data=books, x='year_of_publication', color='lightblue')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=8)
plt.xlabel('Año de Publicación', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.title('Distribución de Años de Publicación de Libros', fontsize=14)
plt.tight_layout()
plt.show()


In [31]:
# Limpieza de valores faltantes en autores y editoriales
books['publisher'] = books['publisher'].str.replace('&', '&', regex=False)
books['publisher'].replace(np.nan, 'Other', inplace=True)
books['book_author'].replace(np.nan, "Unknown", inplace=True)
books.dropna(how='any', axis=0, inplace=True)

In [ ]:
books.head()

### 4.3. Tratamiento de datos de calificaciones

En el dataset de Book-Crossing, las calificaciones de 0 generalmente representan que el usuario ha interactuado con el libro (por ejemplo, lo ha registrado o ha indicado que lo ha leído), pero no ha dado una calificación explícita. Estos 0 no indican necesariamente que al usuario no le gustó el libro, sino que son más bien un indicativo de que no se proporcionó una calificación. En este contexto, pueden ser tratados como valores faltantes (missing), dependiendo de cómo quieras manejar estos datos en tu análisis.

Aquí eliminaremos las calificaciones con valor 0, ya que no aportan información relevante. Eliminar calificaciones sin valor ayuda a concentrar el modelo en interacciones genuinas entre los usuarios y los libros.

In [36]:
# Remover calificaciones con valor 0 (no informativas)
ratings = ratings[ratings['book_rating'] != 0]

In [ ]:
# Visualización de la distribución de calificaciones
plt.figure(figsize=(8, 3))
ax = sns.countplot(data=ratings, x='book_rating', color='lightblue')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=8)
plt.xlabel('Rating del libro', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.title('Distribución de Rating de libros', fontsize=14)
plt.tight_layout()
plt.show()

### 4.4. Unificación de datos


Finalmente, unificaremos los datos de usuarios, calificaciones y libros en un único DataFrame que utilizaremos para construir el modelo de recomendación. Unificar los datos es crucial para poder hacer análisis y recomendaciones basadas en múltiples variables.


In [41]:
# Unir las tablas de usuarios, calificaciones y libros
df_unified = pd.merge(users[['user_id', 'age']], ratings, on='user_id', how='inner')
df_unified = pd.merge(df_unified, books[['isbn', 'book_title']], on='isbn', how='inner')


In [ ]:
df_unified.head()

## 5. Construcción del modelo MPIR


El modelo MPIR recomienda los ítems más populares en base a las calificaciones recibidas. En este caso, los libros que tienen más calificaciones son considerados los más populares. Este enfoque es simple pero efectivo, especialmente en escenarios donde no tenemos información detallada sobre las preferencias individuales de los usuarios.

In [86]:
# Establecer un umbral de rating
rating_threshold = 6

# Filtrar los libros que cumplen con el umbral de rating
filtered_ratings = df_unified[df_unified['book_rating'] >= rating_threshold]

In [111]:
# Agrupar por usuario para contar la cantidad de libros que han leído
books_per_user = filtered_ratings.groupby('user_id')['isbn'].count().reset_index()
books_per_user.rename(columns={'isbn': 'books_count'}, inplace=True)

In [ ]:
# Visualización de la distribución de libros por usuario
plt.figure(figsize=(8, 3))
sns.histplot(books_per_user['books_count'], bins=30, kde=True, color='blue')
plt.xlabel('Cantidad de libros por usuario')
plt.ylabel('Número de usuarios')
plt.title('Distribución de la cantidad de libros vistos por usuario (Rating >= 6)')
plt.show()

In [113]:
# Filtrar para quedarse solo con usuarios que han visto al menos 5 libros
books_per_user_filtered = books_per_user[books_per_user['books_count'] >= 5]

# Calcular el percentil 95 de la distribución de libros vistos
percentil_95 = books_per_user_filtered['books_count'].quantile(0.95)

# Filtrar para quedarse solo con usuarios que han visto hasta el percentil 95
final_filtered_users = books_per_user_filtered[books_per_user_filtered['books_count'] <= percentil_95]


In [ ]:
# Visualización de la distribución de libros por usuario
plt.figure(figsize=(8, 3))
sns.histplot(final_filtered_users['books_count'], bins=30, kde=True, color='blue')
plt.xlabel('Cantidad de libros por usuario')
plt.ylabel('Número de usuarios')
plt.title('Distribución de la cantidad de libros vistos por usuario (Rating >= 6)')
plt.show()

In [ ]:
# Filtrar el DataFrame original para quedarse solo con los usuarios seleccionados
filtered_ratings_final = filtered_ratings[filtered_ratings['user_id'].isin(final_filtered_users['user_id'])]
filtered_ratings_final.head()

In [119]:
# División de los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

train_user_ids, test_user_ids = train_test_split(final_filtered_users['user_id'].unique(), train_size=0.7, random_state=123)


In [226]:
# Filtrar los datos para obtener el conjunto de entrenamiento y prueba
train = filtered_ratings_final[filtered_ratings_final['user_id'].isin(train_user_ids)]
test = filtered_ratings_final[filtered_ratings_final['user_id'].isin(test_user_ids)]


In [ ]:
print(f'Tamaño del conjunto de entrenamiento: {len(train_user_ids)}')
print(f'Tamaño del conjunto de prueba: {len(test_user_ids)}')

In [121]:
# Verificar que no haya intersección entre los user_ids de train y test
assert len(set(train['user_id']).intersection(set(test['user_id']))) == 0, "Hay user_ids que se repiten en train y test"


In [124]:
# Calcular la popularidad de los libros en el conjunto de entrenamiento
most_popular = train.groupby('book_title')[['isbn']].count().reset_index()
most_popular.rename(columns={'isbn': 'popularity'}, inplace=True)
most_popular.sort_values(by='popularity', ascending=False, inplace=True)

In [ ]:
# Mostrar los libros más populares
top10 = most_popular.head(10)
top10

In [ ]:
# Crear un gráfico de barras
plt.figure(figsize=(10, 4))
plt.barh(top10['book_title'], top10['popularity'], color='skyblue')
plt.xlabel('Popularity', fontsize=12)
plt.ylabel('Book Title', fontsize=12)
plt.title('Top 10 Most Popular Books', fontsize=14)
plt.gca().invert_yaxis()  # Invertir el eje y para que el libro más popular esté en la parte superior
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular el porcentaje de popularidad
top10['popularity_percent'] = (top10['popularity'] / len(train_user_ids)) * 100


# Crear un gráfico de barras con porcentajes
plt.figure(figsize=(10, 4))
plt.barh(top10['book_title'], top10['popularity_percent'], color='skyblue')
plt.xlabel('Popularity (%)', fontsize=12)
plt.ylabel('Book Title', fontsize=12)
plt.title('Top 10 Most Popular Books by Percentage', fontsize=14)
plt.gca().invert_yaxis()  # Invertir el eje y para que el libro más popular esté en la parte superior
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()

# Mostrar el gráfico
plt.show()


## 6. Evaluación del modelo (Hit Rate, Precisión y Diversidad)


En esta sección, evaluaremos la efectividad del recomendador utilizando tres métricas: <b>Hit Rate, Precisión y Diversidad.</b> Estas métricas nos ayudarán a entender la calidad de las recomendaciones.

- 6.1. Hit Rate: Mide la proporción de usuarios que recibieron al menos una recomendación relevante.
- 6.2. Precisión: Mide la proporción de recomendaciones relevantes entre el total de recomendaciones.
- 6.3. Diversidad: Mide cuán variadas son las recomendaciones dentro de una lista.


### 6.1. Aplicación de recomendaciones

Las recomendaciones se basan en los libros más populares Asignaremos los libros más populares como recomendaciones para el conjunto de prueba.


In [ ]:
top10

In [137]:
test['rec_1'] = top10.book_title.tolist()[0]
test['rec_2'] = top10.book_title.tolist()[1]
test['rec_3'] = top10.book_title.tolist()[2]
test['rec_4'] = top10.book_title.tolist()[3]
test['rec_5'] = top10.book_title.tolist()[4]
test['rec_6'] = top10.book_title.tolist()[5]
test['rec_7'] = top10.book_title.tolist()[6]
test['rec_8'] = top10.book_title.tolist()[7]
test['rec_9'] = top10.book_title.tolist()[8]
test['rec_10'] = top10.book_title.tolist()[9]


### 6.2. Cálculo de Hit Rate

Hit Rate mide la proporción de usuarios que recibieron al menos una recomendación relevante.

In [ ]:
# Cálculo del "hit" para cada usuario basado en si alguna de las recomendaciones coincide con un libro que le gustó
test['hit'] = (
    (test.book_title == test.rec_1) |
    (test.book_title == test.rec_2) |
    (test.book_title == test.rec_3) |
    (test.book_title == test.rec_4) |
    (test.book_title == test.rec_5) |
    (test.book_title == test.rec_6) |
    (test.book_title == test.rec_7) |
    (test.book_title == test.rec_8) |
    (test.book_title == test.rec_9) |
    (test.book_title == test.rec_10)
)

# Agregación por usuario: verificar si hubo algún "hit" para cada usuario
user_hits = test.groupby('user_id')['hit'].max().reset_index()

# Cálculo del Hit Rate: proporción de usuarios con al menos un "hit"
hit_rate = round(user_hits['hit'].mean() * 100, 1)
print(f'Hit Rate: {hit_rate}%')

### 6.3. Cálculo de Precisión


In [ ]:
# Precisión
test_cliente = test.groupby('user_id').aggregate({'hit': 'sum'}).reset_index()
precision = round(test_cliente['hit'].mean() / 5 * 100, 1)
print(f'Semi-Personalized Precision: {precision}%')

In [ ]:
# Contar cuántos "hits" hubo para cada usuario, es decir, cuántas de las recomendaciones coincidieron con un libro que le gustó
test['hits'] = (
    (test.book_title == test.rec_1).astype(int) +
    (test.book_title == test.rec_2).astype(int) +
    (test.book_title == test.rec_3).astype(int) +
    (test.book_title == test.rec_4).astype(int) +
    (test.book_title == test.rec_5).astype(int) +
    (test.book_title == test.rec_6).astype(int) +
    (test.book_title == test.rec_7).astype(int) +
    (test.book_title == test.rec_8).astype(int) +
    (test.book_title == test.rec_9).astype(int) +
    (test.book_title == test.rec_10).astype(int)
)

# Agrupar por usuario y sumar los "hits" por usuario
user_hits = test.groupby('user_id')['hits'].sum().reset_index()
user_hits.sort_values(by = 'hits', ascending = False).head()


In [ ]:
# Calcular la precisión por usuario (número de hits dividido entre el número de recomendaciones)
user_hits['precision'] = user_hits['hits'] /10  # número total de recomendaciones que diste

# Calcular la precisión promedio para todos los usuarios
precision = round(user_hits['precision'].mean() * 100, 1)
print(f'Precision: {precision}%')

In [ ]:
# Visualización de la distribución de libros por usuario
plt.figure(figsize=(8, 3))
sns.histplot(user_hits['hits'], color='blue')
plt.xlabel('Cantidad de libros que hicieron hit')
plt.ylabel('Número de usuarios')
plt.title('Distribución de la cantidad de libros que hicieron hit')
plt.show()

### 6.4. Cálculo de Diversidad


La Diversidad Intra-Lista (ILD) mide cuán variadas son las recomendaciones dentro de una lista. Se utiliza una métrica de similitud para evaluar esto.

In [ ]:
top10.book_title.tolist()

In [ ]:
books

In [ ]:
top10_titles

In [ ]:
# Suponiendo que tienes un DataFrame `df_unified` que incluye las características de los libros
top10_titles = top10.book_title.tolist()

# Filtrar los libros de la lista de recomendaciones top10 para obtener sus características
recommended_books = books[books['book_title'].isin(top10_titles)][['book_title', 'book_author', 'year_of_publication', 'publisher']]
recommended_books = recommended_books.drop_duplicates(subset = 'book_title', keep='first')
recommended_books


In [185]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

# 1. Convertir las características de texto a representaciones numéricas
vectorizer_author = CountVectorizer()
vectorizer_publisher = CountVectorizer()

author_features = vectorizer_author.fit_transform(recommended_books['book_author']).toarray()
publisher_features = vectorizer_publisher.fit_transform(recommended_books['publisher']).toarray()


In [187]:
# 2. Calcular la antigüedad en años (2024 - year_of_publication)
recommended_books['antiguedad'] = 2024 - recommended_books['year_of_publication']

# Escalar la antigüedad para que esté en una escala comparable
scaler = StandardScaler()
antiguedad_features = scaler.fit_transform(recommended_books[['antiguedad']])


In [188]:
# 3. Crear una matriz de características combinadas
combined_features = np.hstack([author_features, antiguedad_features, publisher_features])


In [196]:
# 4. Calcular la similitud coseno entre los libros en la lista de recomendaciones
similarity_matrix = cosine_similarity(combined_features)

#### Contexto de ILD:

ILD (Intra-List Diversity) es una métrica que varía entre 0 y 1, donde:
- 0 indica que todas las recomendaciones son muy similares entre sí.
- 1 indica que todas las recomendaciones son completamente diferentes entre sí.

In [ ]:
# 5. Calcular la Diversidad Intra-Lista (ILD)
n_items = len(top10_titles)
ild = 1 - np.sum(similarity_matrix) / (n_items * (n_items - 1))

print(f"Intra-List Diversity (ILD): {ild:.2f}")

### Interpretación:

ILD = 0.88 significa que la lista de recomendaciones es  diversa. La similitud promedio entre los ítems en la lista es baja, lo que indica que los libros recomendados tienen características bastante diferentes entre sí.

## 7. Semi-Personalización del Recomendador


En esta sección, vamos a introducir una semi-personalización del sistema de recomendación utilizando la variable de edad de los usuarios. Dividiremos a los usuarios en rangos de edad y recomendaremos libros populares dentro de cada grupo. Este enfoque permite ofrecer recomendaciones más relevantes sin requerir información detallada sobre cada usuario.


### 7.1. Discretización de la variable de edad


In [ ]:
users[users.user_id.isin(train_user_ids)]

In [221]:
# Imputar valores NaN en 'age' con la mediana (o podrías usar la media, según lo prefieras)
users['age'].fillna(users['age'].median(), inplace=True)

# Vamos a agrupar las edades en cinco rangos utilizando KBinsDiscretizer.

from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy="quantile")
discretizer = discretizer.fit(np.array(users[users.user_id.isin(train_user_ids)]['age']).reshape(-1, 1))


In [ ]:
# Aplicar la transformación a todo el conjunto de datos
users['age_rango'] = discretizer.transform(np.array(users['age']).reshape(-1, 1))
users.head()

In [223]:
# Asumiendo que discretizer ya ha sido ajustado y aplicado
import numpy as np

# Definir los límites de los bins (rangos de edad) basados en los cortes realizados por el discretizer
bin_edges = discretizer.bin_edges_[0]

# Crear las etiquetas de los rangos
age_labels = [f"{int(bin_edges[i])}-{int(bin_edges[i+1])-1}" for i in range(len(bin_edges)-1)]

# Asignar los valores discretizados a sus correspondientes etiquetas
users['age_rango_desc'] = pd.cut(users['age'], bins=bin_edges, labels=age_labels, include_lowest=True, right=False)

# Verificar la transformación
users[['age', 'age_rango', 'age_rango_desc']].head()


,age,age_rango,age_rango_desc
0,32.0,2.0,32-41
1,18.0,0.0,2-26
2,32.0,2.0,32-41
3,17.0,0.0,2-26
4,32.0,2.0,32-41


In [227]:
# Filtrar el DataFrame de entrenamiento para trabajar solo con los usuarios de train
train = filtered_ratings_final[filtered_ratings_final['user_id'].isin(train_user_ids)]
train = pd.merge(train, users[['user_id', 'age_rango', 'age_rango_desc']], on='user_id', how='left')

# Calcular la popularidad de los libros dentro de cada rango de edad en el conjunto de entrenamiento
most_popular_age = train.groupby(['book_title', 'age_rango', 'age_rango_desc'])[['isbn']].count().reset_index()
most_popular_age.rename(columns={'isbn': 'popularity'}, inplace=True)
most_popular_age.sort_values(by='popularity', ascending=False, inplace=True)


In [ ]:
# Mostrar los libros más populares en cada rango de edad
for i in range(5):
    print(f"Top 5 libros para el rango de edad {i}:")
    print(most_popular_age[most_popular_age['age_rango'] == i].head(5), '\n')


### 7.3. Aplicar recomendaciones semi-personalizadas


Ahora, aplicaremos las recomendaciones basadas en el rango de edad del usuario.


In [234]:
# Incorporar el rango de edad al conjunto de prueba
test = filtered_ratings_final[filtered_ratings_final['user_id'].isin(test_user_ids)]
test = pd.merge(test, users[['user_id', 'age_rango', 'age_rango_desc']], on='user_id', how='left')

In [236]:
# Crear el DataFrame de recomendaciones semi-personalizadas basado en los libros más populares para cada rango de edad
recom = pd.DataFrame({
    'age_rango': [0, 1, 2, 3, 4],
    'rec_1': [most_popular_age[most_popular_age.age_rango == i].book_title.tolist()[0] if len(most_popular_age[most_popular_age.age_rango == i].book_title.tolist()) > 0 else "No disponible" for i in range(5)],
    'rec_2': [most_popular_age[most_popular_age.age_rango == i].book_title.tolist()[1] if len(most_popular_age[most_popular_age.age_rango == i].book_title.tolist()) > 1 else "No disponible" for i in range(5)],
    'rec_3': [most_popular_age[most_popular_age.age_rango == i].book_title.tolist()[2] if len(most_popular_age[most_popular_age.age_rango == i].book_title.tolist()) > 2 else "No disponible" for i in range(5)],
    'rec_4': [most_popular_age[most_popular_age.age_rango == i].book_title.tolist()[3] if len(most_popular_age[most_popular_age.age_rango == i].book_title.tolist()) > 3 else "No disponible" for i in range(5)],
    'rec_5': [most_popular_age[most_popular_age.age_rango == i].book_title.tolist()[4] if len(most_popular_age[most_popular_age.age_rango == i].book_title.tolist()) > 4 else "No disponible" for i in range(5)]
})

# Fusionar las recomendaciones con el conjunto de prueba
test = pd.merge(test, recom, on='age_rango', how='left')


In [ ]:
test.head()

### 7.4. Evaluar el modelo semi-personalizado


Evaluaremos la efectividad del sistema semi-personalizado utilizando las mismas métricas que antes.


In [ ]:
# Hit Rate


In [ ]:
# Precisión





In [ ]:
# Diversidad



## 8. Desafíos Adicionales


#### Explorar Otros Ejes de Personalización

Además de la edad, ¿qué otras variables demográficas podrías utilizar para personalizar las recomendaciones? Por ejemplo, ¿qué impacto tendría la ubicación geográfica en las recomendaciones?


#### Evaluar el Efecto de la Personalización en la Diversidad
¿Cómo afecta la personalización la diversidad de las recomendaciones? Intenta balancear la personalización con la diversidad y mide el impacto en la precisión y el hit rate.



## 9. Conclusiones

- En este laboratorio, construimos un sistema de recomendación no personalizado utilizando el enfoque de ítems más populares (MPIR). Evaluamos la efectividad del sistema utilizando las métricas de Hit Rate, Precisión y Diversidad, y encontramos que si bien es un sistema sencillo de implementar, tiene limitaciones en cuanto a la personalización y la diversidad de recomendaciones.

- Posteriormente, introducimos una semi-personalización basada en la edad de los usuarios, lo que mejoró ligeramente la relevancia de las recomendaciones para diferentes grupos demográficos. Este enfoque es un primer paso hacia sistemas de recomendación más sofisticados que pueden ofrecer recomendaciones más personalizadas y variadas.

- Finalmente, propusimos algunos desafíos adicionales para que los estudiantes exploren más allá del laboratorio, incluyendo la implementación de un filtro colaborativo, la exploración de otros ejes de personalización y el análisis del balance entre personalización y diversidad.

- Con este conocimiento, estarán mejor preparados para diseñar e implementar sistemas de recomendación que sean efectivos y relevantes para los usuarios, entendiendo las trade-offs y decisiones clave que deben tomarse en su construcción.

---
## Gracias por completar este laboratorio!